In [ ]:
import os
import cv2
import time

from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Egor\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [1]:
import torch
print(torch.__version__)          # Должно быть: 2.x.x+cuXXX (например, 2.3.0+cu118)
print(torch.cuda.is_available())  # Должно быть True

2.5.1+cu121
True


In [25]:
from pathlib import Path

images_path = Path('C:/Users/Egor/Desktop/ss/AI-Flow-Detecting/ai-core/dataset/train_2/obj_train_data')

for img_file in images_path.glob("*.jpg"):
    txt_file = img_file.with_suffix('.txt')
    if not txt_file.exists():
        img_file.unlink()  # Удаление .jpg без .txt
        print(f"Удалено: {img_file}")



In [ ]:
from ultralytics import YOLO
import yaml
import os
from pathlib import Path
import torch

device = 0 if torch.cuda.is_available() else 'cpu'
print(f"Используется: {device}")

model = YOLO('yolov10m.pt').to(device)

model.train(
    data='data.yaml',
    epochs=40,
    imgsz=640,
    batch=16,
    device=device,
    name='yolov10n_bus_stop_finetuned',
    augment=True,
    patience=15,
    optimizer='AdamW',
    lr0=1e-3,
    iou=0.5,
    conf=0.25,
    project='runs/train',
    workers=4,
    cache=True,
    amp=True,
    #epochs=100,               # Больше эпох
    #batch=32,                # Увеличенный batch
    #hsv_h=0.015,             # Настройки HSV
    #hsv_s=0.7,
    #hsv_v=0.4,
    #translate=0.2,           # Сдвиг
    #fliplr=0.5,              # Отражение
    #mosaic=1.0,              # Мозаика
    #mixup=0.2,              # Mixup
    #patience=10,             # Ранняя остановка
    #lr0=3e-4,               # Скорость обучения
    #lrf=0.01,               # Final LR (1% от lr0)
    #iou=0.7,                # Более строгий IoU
    #rect=True,              # Прямоугольное обучение
    #cache='ram',            # Кеширование в RAM
    #amp=True,               # Mixed precision
    #plots=True,             # Графики в TensorBoard
)

model.save('models/yolov10n_best.pt')
print("✅ Модель дообучена и сохранена.")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4196264475.py, line 28)

In [ ]:
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# --- 1. Загрузка модели ---
model = YOLO("runs/train/yolov10n_bus_stop_finetuned24/weights/best.pt")

# --- 2. Тестирование на изображениях ---
print("🔍 Запуск предсказания на тестовых данных...")
results = model.predict(
    source="dataset/test/images",
    imgsz=640,
    conf=0.25,
    save=True,
    save_txt=True,
    name="predict_test"  # папка для сохранения результатов
)

# --- 3. Оценка метрик на валидационном наборе ---
print("\n📊 Оценка метрик модели...")
metrics = model.val(
    data="data.yaml",
    split="val",
    batch=16,
    imgsz=640,
    conf=0.25,
    iou=0.6,
    device="0",
    plots=True,
    name="val_metrics"
)

# Вывод ключевых метрик
print(f"\n📌 Итоговые метрики:")
print(f"mAP@0.5: {metrics.box.map:.3f}")
print(f"mAP@0.5:0.95: {metrics.box.map50:.3f}")

# --- 4. Анализ графиков обучения ---
try:
    print("\n📈 Визуализация кривых обучения...")
    results_csv = pd.read_csv("runs/train/yolov10n_bus_stop_finetuned24/results.csv")
    
    plt.figure(figsize=(15, 10))
    
    # График Precision-Recall
    plt.subplot(2, 2, 1)
    plt.plot(results_csv["epoch"], results_csv["metrics/precision(B)"], label="Precision")
    plt.plot(results_csv["epoch"], results_csv["metrics/recall(B)"], label="Recall")
    plt.title("Precision & Recall")
    plt.legend()
    
    # График mAP
    plt.subplot(2, 2, 2)
    plt.plot(results_csv["epoch"], results_csv["metrics/mAP50(B)"], label="mAP@0.5")
    plt.plot(results_csv["epoch"], results_csv["metrics/mAP50-95(B)"], label="mAP@0.5:0.95")
    plt.title("mAP Metrics")
    plt.legend()
    
    # График потерь при обучении
    plt.subplot(2, 2, 3)
    plt.plot(results_csv["epoch"], results_csv["train/box_loss"], label="Box Loss")
    plt.plot(results_csv["epoch"], results_csv["train/cls_loss"], label="Cls Loss")
    plt.title("Training Losses")
    plt.legend()
    
    # График потерь на валидации
    plt.subplot(2, 2, 4)
    plt.plot(results_csv["epoch"], results_csv["val/box_loss"], label="Val Box Loss")
    plt.plot(results_csv["epoch"], results_csv["val/cls_loss"], label="Val Cls Loss")
    plt.title("Validation Losses")
    plt.legend()
    
    plt.tight_layout()
    plt.show()

except FileNotFoundError:
    print("⚠️ Файл results.csv не найден!")

# --- 5. Визуализация примеров предсказаний ---
print("\n🖼️ Примеры предсказаний:")
try:
    # Показываем первое сохранённое предсказание
    pred_image = Image.open("runs/detect/predict_test/image0.jpg")
    plt.figure(figsize=(10, 8))
    plt.imshow(pred_image)
    plt.axis('off')
    plt.title("Пример предсказания")
    plt.show()
except:
    print("⚠️ Не удалось загрузить пример предсказания")

print("\n✅ Анализ завершён!")

🔍 Запуск предсказания на тестовых данных...

image 1/1 c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\ai-core\dataset\test\images\20250810_130413.jpg: 384x640 22 humans, 20.7ms
Speed: 1.9ms preprocess, 20.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\runs\detect\predict_test6
1 label saved to c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\runs\detect\predict_test6\labels

📊 Оценка метрик модели...
Ultralytics 8.3.177  Python-3.12.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.00.0 ms, read: 3026.6560.0 MB/s, size: 489.1 KB)


val: Scanning C:\Users\Egor\Desktop\ss\AI-Flow-Detecting\ai-core\dataset\train_1\obj_train_data.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]


                   all         52        109      0.153     0.0275     0.0658      0.019
                 human         52        109      0.153     0.0275     0.0658      0.019
Speed: 1.6ms preprocess, 14.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\runs\detect\val_metrics6

📌 Итоговые метрики:
mAP@0.5: 0.019
mAP@0.5:0.95: 0.066

📈 Визуализация кривых обучения...


<Figure size 1500x1000 with 4 Axes>


🖼️ Примеры предсказаний:
⚠️ Не удалось загрузить пример предсказания

✅ Анализ завершён!


In [30]:
metrics = model.val(
    data="data.yaml",
    imgsz=640,
    conf=0.25
)

Ultralytics 8.3.177  Python-3.12.5 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 142.987.0 MB/s, size: 501.6 KB)


val: Scanning C:\Users\Egor\Desktop\ss\AI-Flow-Detecting\ai-core\dataset\train_1\obj_train_data.cache... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]


                   all         52        109      0.153     0.0275     0.0658      0.019
                 human         52        109      0.153     0.0275     0.0658      0.019
Speed: 1.5ms preprocess, 12.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to c:\Users\Egor\Desktop\ss\AI-Flow-Detecting\runs\detect\val2


# Подсчёт пассажиров в видеопотоке

In [33]:
# from ultralytics import YOLO

# model = YOLO('runs/train/yolov10n_bus_stop_finetuned24/weights/best.pt')

# # Запуск трекинга на видео
# model.track(
#     source='video.mp4',
#     tracker='bytetrack.yaml',  # можно 'deepsort.yaml'
#     conf=0.5,
#     save=True
# )


In [35]:
# # упрощённый фрагмент
# if hasattr(res, 'masks') and res.masks is not None:
#     im_with_masks = res.masks.render(res.orig_img)  # иногда работает напрямую
#     frame = im_with_masks


In [34]:
# def dice_coef(gt_mask, pred_mask):
#     gt = gt_mask.astype(bool)
#     pr = pred_mask.astype(bool)
#     inter = (gt & pr).sum()
#     return 2*inter / (gt.sum() + pr.sum() + 1e-6)
